In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from agentarena.services.queue_service import QueueService
from agentarena.models.job import Job

from litequeue import LiteQueue
    
def get_q(filename: str, memory: bool = False) -> LiteQueue:
    dbfile = ":memory:" if memory else filename

    return LiteQueue(dbfile)

q = QueueService("<projectroot>", "jobs.db", get_q)
q.drain()

j1 = Job(
    id="test 1",
    command="test",
    payload="I am a test payload",
    caller="test caller"
)
j2 = Job(
    id="test 2",
    command="test",
    payload="I am a test payload",
    caller="test caller"
)
q.send_job(j1)
q.send_job(j2)

print(q.get_job())
print('---')

j1, t1 = q.get_job()
print(j1)
q.done(t1)

j2, t2 = q.get_job()
q.done(t2)
print(j2)




2025-05-03 18:39:36 [info     ] Setting up Job DB at jobs.db   db=jobs.db module=queue_service
2025-05-03 18:39:36 [warning  ] Draining the queue             db=jobs.db module=queue_service
(Job(id='test 1', active=True, created_at=datetime.datetime(2025, 5, 3, 18, 39, 36, 842705), updated_at=None, deleted_at=None, command='test', payload='I am a test payload', caller='test caller', status='IDLE'), Message(data='{"id":"test 1","active":true,"created_at":"2025-05-03T18:39:36.842705","updated_at":null,"deleted_at":null,"command":"test","payload":"I am a test payload","caller":"test caller","status":"IDLE"}', message_id='06816c55-8d7b-7e7c-8000-2687ea3b5a6a', status=1, in_time=1746322776842765462, lock_time=1746322776843062865, done_time=None))
---
id='test 2' active=True created_at=datetime.datetime(2025, 5, 3, 18, 39, 36, 842891) updated_at=None deleted_at=None command='test' payload='I am a test payload' caller='test caller' status='IDLE'
2025-05-03 18:39:36 [info     ] Task done: task

In [4]:
from sqlite_utils import Database
from datetime import datetime

db = Database("test.db")
db["ex1"].insert({"id": "test2", "name": "test", "timestamp": datetime.now()}, pk="id")
print (db["ex1"].get("test2"))

{'id': 'test2', 'name': 'test', 'timestamp': '2025-04-28T21:01:59.552417'}


In [ ]:
print (db['ex1'].schema)
print (db['ex1'].get("test2")["timestamp"])

CREATE TABLE [ex1] (
   [id] TEXT PRIMARY KEY,
   [name] TEXT,
   [timestamp] TEXT
)
2025-04-28T21:01:59.552417


In [13]:
author = db['authors'].insert({
    "name": "Jeb Doe",
    "id": "jeb"
}, pk="id")

posts = db['posts'].create({
    "id": str,
    "message": str,
    "author_id": str,
}, replace=True, pk="id", foreign_keys=["author_id"])


In [14]:
posts.insert({
    "message": "Hello world "
})

<Table posts (id, message, author_id)>

In [22]:
# testing a bad id
db.execute("PRAGMA foreign_keys = ON")

posts.insert({
    "message": "Hello world 3",
    "author_id": "bad_id"
}, foreign_keys=["author_id"])


IntegrityError: FOREIGN KEY constraint failed

In [21]:
posts.schema
print ([row for row in posts.rows])


[{'id': None, 'message': 'Hello world ', 'author_id': None}, {'id': None, 'message': 'Hello world 2', 'author_id': 'bad_id'}, {'id': None, 'message': 'Hello world 3', 'author_id': 'bad_id'}]


In [24]:
# Is there a FK inferred automatically?

fktest = db["fktest"].insert({
    "id": 1,
    "message": "test",
    "author_id": "john"
})



In [25]:
fktest.schema

'CREATE TABLE [fktest] (\n   [id] INTEGER,\n   [message] TEXT,\n   [author_id] TEXT\n)'

In [27]:
# can I insert with a null FK?

db['insertit'].insert({
    "id": "foo"
}, foreign_keys=[])


<Table insertit (id)>

In [32]:
db['insertit'].schema


'CREATE TABLE [insertit] (\n   [id] TEXT\n)'

In [ ]:
from agentarena.models.strategy import StrategyDTO

s = StrategyDTO.model_validate_json('''
                                 {
  "name": "Aggressive Player",
  "personality": "Aggressive",
  "instructions": "Always attack when possible.",
  "description": "A strategy for aggressive play style.",
  "role": "player"
}
''')

print(s)

id=None active=True created_at=None updated_at=None deleted_at=None FOREIGN_KEYS=[] name='Aggressive Player' personality='Aggressive' instructions='Always attack when possible.' description='A strategy for aggressive play style.' role=<StrategyType.PLAYER: 'player'>


In [41]:
db = Database("test.db")
text = s.model_dump_json()
print(text)
db["strategies"].insert(text, pk="id")
print(db["strategies"].get(s.id))

{"id":null,"active":true,"created_at":null,"updated_at":null,"deleted_at":null,"FOREIGN_KEYS":[],"name":"Aggressive Player","personality":"Aggressive","instructions":"Always attack when possible.","description":"A strategy for aggressive play style.","role":"player"}


AttributeError: 'str' object has no attribute 'keys'

In [ ]:
# making a map from a list of dicts

from pydantic import BaseModel
import json

class Test(BaseModel):
    id: str
    name: str
    
t1 = Test(id="1", name="test1")
t2 = Test(id="2", name="test2")

l = [t1, t2]
d = {t.id: t for t in l}

print(d)


{'1': Test(id='1', name='test1'), '2': Test(id='2', name='test2')}
